In [179]:
import json
# from string import maketrans
from sklearn.feature_extraction.text import TfidfVectorizer 
import re
import ftfy as ft
import time
import numpy as np
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from scipy.sparse import csr_matrix
import sparse_dot_topn.sparse_dot_topn as ct
import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

t=open(r"C:\dev\personal\Web Scraping\projects\tesco\tesco_072720.json")
a=open(r"C:\dev\personal\Web Scraping\projects\asda\asda_072720.json")
s=open(r"C:\dev\personal\Web Scraping\projects\sainsburys\sainsburys_072720.json")
w=open(r"C:\dev\personal\Web Scraping\projects\waitrose\waitrose_072720.json")
m=open(r"C:\dev\personal\Web Scraping\projects\morrisons\morrisons_072720.json")

C:\Users\sujithk\AppData\Local\Continuum\anaconda3\envs\Virtual_Env\lib\site-packages\ipykernel_launcher.py:16: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  app.launch_new_instance()


In [180]:
tesco_price=json.load(t)
# print(tesco_price)
ctr=1
tesco_prod_name=[]
for i in tesco_price:
    tesco_prod_name.insert(ctr,i['tesco_prod_name'])
    ctr+=1

#print(tesco_prod_name)
t.close()

asda_price=json.load(a)
# print(asda_price)
ctr=1
asda_prod_name=[]
for i in asda_price:
    asda_prod_name.insert(ctr,i['asda_prod_name'])
    ctr+=1

#print(asda_prod_name)
a.close()

morrisons_price=json.load(m)
# print(morrisons_price)
ctr=1
morrisons_prod_name=[]
for i in morrisons_price:
    morrisons_prod_name.insert(ctr,i['morrisons_prod_name'])
    ctr+=1

#print(morrisons_prod_name)
m.close()

waitrose_price=json.load(w)
# print(waitrose_price)
ctr=1
waitrose_prod_name=[]
for i in waitrose_price:
    waitrose_prod_name.insert(ctr,i['waitrose_prod_name'])
    ctr+=1

#print(waitrose_prod_name)
w.close()

sainsburys_price=json.load(s)
# print(sainsburys_price)
ctr=1
sainsburys_prod_name=[]
for i in sainsburys_price:
    sainsburys_prod_name.insert(ctr,i['sainsburys_prod_name'])
    ctr+=1

#print(sainsburys_prod_name)
s.close()



In [218]:
def uniq(list1):
    list_set=set(list1)
    unique_list=list(list_set)
    return unique_list

tesco_prod_name_uniq=np.array(uniq(tesco_prod_name))
asda_prod_name_uniq=np.array(uniq(asda_prod_name))
morrisons_prod_name_uniq=np.array(uniq(morrisons_prod_name))
sainsburys_prod_name_uniq=np.array(uniq(sainsburys_prod_name))
waitrose_prod_name_uniq=np.array(uniq(waitrose_prod_name))

prim_vec_count=len(tesco_prod_name_uniq)
morrisons_trg_vec_count=len(morrisons_prod_name_uniq)
waitrose_trg_vec_count=len(waitrose_prod_name_uniq)
sainsburys_trg_vec_count=len(sainsburys_prod_name_uniq)
asda_trg_vec_count=len(asda_prod_name_uniq)


print("Shape of tesco pre deduping")
print(len(tesco_prod_name))
print("Shape and count of tesco post deduping")
print(prim_vec_count)

print("Shape of asda pre deduping")
print(len(asda_prod_name))
print("Shape and count of asda post deduping")
print(asda_trg_vec_count)

print("Shape of morrisons pre deduping")
print(len(morrisons_prod_name))
print("Shape and count of morrisons post deduping")
print(morrisons_trg_vec_count)

print("Shape of sainsburys pre deduping")
print(len(sainsburys_prod_name))
print("Shape and count of sainsburys post deduping")
print(sainsburys_trg_vec_count)

print("Shape of waitrose pre deduping")
print(len(waitrose_prod_name))
print("Shape and count of waitrose post deduping")
print(waitrose_trg_vec_count)


#test_vector=np.array(tesco_prod_name_uniq+morrisons_prod_name_uniq)


Shape of tesco pre deduping
204
Shape and count of tesco post deduping
204
Shape of asda pre deduping
248
Shape and count of asda post deduping
61
Shape of morrisons pre deduping
340
Shape and count of morrisons post deduping
340
Shape of sainsburys pre deduping
179
Shape and count of sainsburys post deduping
179
Shape of waitrose pre deduping
135
Shape and count of waitrose post deduping
135


In [236]:
frtoeng = "".maketrans("àâçéèêîôùû", "aaceeeiouu")

def cleanup(string):
    string = ft.fix_text(string) # fix text encoding issues
    string = string.translate(frtoeng)
    string = string.encode("ascii", errors="ignore").decode() #remove non ascii chars
    string = string.lower() #make lower case
    chars_to_remove = [")","(",".","|","[","]","{","}","'"," "]
    rx = '[' + re.escape(''.join(chars_to_remove)) + ']'
    string = re.sub(rx, '', string) #remove the list of chars defined above
    string = string.replace('&', '')
    string = string.replace('and', '')
    string = string.replace('ml', '')
    string = string.replace('Ml', 'ml')
    string = string.replace(',', '')
    string = string.replace('-', '')
    #string="".join(sorted(string))
    #string = string.replace('shampoo', '')
    #string = string.title() # normalise case - capital at start of each word
    string = re.sub(' +',' ',string).strip() # get rid of multiple spaces and replace with a single space
    # string = ' '+ string +' ' # pad names for ngrams...
    string = re.sub(r'[,-./]|\sBD',r'', string)
    return string

f= lambda x: cleanup(x)
morrisons_prod_name_uniq_cleaned= np.array([f(xi) for xi in morrisons_prod_name_uniq])
waitrose_prod_name_uniq_cleaned= np.array([f(xi) for xi in waitrose_prod_name_uniq])
sainsburys_prod_name_uniq_cleaned= np.array([f(xi) for xi in sainsburys_prod_name_uniq])


#Test Cases
## Case I
print(cleanup("L'OrÃƒÂ©al Elvive Extraordinary Oil Shampoo Dry Hair  250ml"))
print("L'OrÃƒÂ©al Elvive Extraordinary Oil Shampoo Dry Hair  250ml"["L'OrÃƒÂ©al Elvive Extraordinary Oil Shampoo Dry Hair  250ml".rstrip().rfind(" "):-2])
j="L'OrÃƒÂ©al Elvive Extraordinary Oil Shampoo Dry Hair  250ml"["L'OrÃƒÂ©al Elvive Extraordinary Oil Shampoo Dry Hair  250ml".rstrip().rfind(" "):-2]

## Case II

print("".join(sorted(cleanup("Herbal Essences Bio. Renew Argan Shampoo 400Ml"))))
print("".join(sorted(cleanup("Herbal Essences Bio Renew Shampoo Coconut Milk Hydrate 400ml"))))
print("".join(sorted(cleanup("Herbal Essences Bio Renew Shampoo Argan Oil of Morocco 400ml")))) #ideally this one should be selected as the right match


## Case III


print("".join(sorted(cleanup("Alberto Balsam Apple Shampoo 350Ml"))))

k=[x for x in morrisons_prod_name if x.find("Alberto Balsam")!=-1]
l=np.array(uniq(k))
print(l)
ratio = process.extract(cleanup("Alberto Balsam Apple Shampoo 350Ml"), morrisons_prod_name_uniq_cleaned, limit=1, scorer=fuzz.partial_token_set_ratio)
print(ratio[0][0])
res=np.where(morrisons_prod_name_uniq_cleaned==ratio[0][0])
print(morrisons_prod_name_uniq[res[0][0]])
# k=np.array([x for x in morrisons_prod_name if x.find(j)!=-1])
# print(k)

lorealelviveextraordinaryoilshampoodryhair250
 250
004aaaabbceeeeeeghhilmnnnoooprrrssssw
004aaabbcccdeeeeeeehhhiikllmmnnnoooooprrrssssttuwy
004aaaabbccceeeeeefghhiillmmnnnooooooooprrrrssssw
035aaaaabbeehlllmmoooppprsst
['Alberto Balsam Tea Tree Tingle Conditioner  350ml'
 'Alberto Balsam Juicy Green Apple Conditioner  350ml'
 'Alberto Balsam Raspberry Shampoo 350ml'
 'Alberto Balsam Restore Shampoo 250ml'
 'Alberto Balsam Energize Conditioner 250ml'
 'Alberto Balsam Energize Shampoo 250ml'
 'Alberto Balsam Coconut & Lychee Nourishing Shampoo 350ml'
 'Alberto Balsam Tea Tree Tingle Shampoo  350ml'
 'Alberto Balsam Juicy Green Apple Shampoo  350ml'
 'Alberto Balsam Raspberry Conditioner 350ml'
 'Alberto Balsam Coconut & Lychee Conditioner  350ml'
 'Alberto Balsam Restore Conditioner 250ml']
albertobalsamraspberryshampoo350
Alberto Balsam Raspberry Shampoo 350ml


In [220]:

matched_prod = []
similarity = []


# print(morrisons_prod_name_uniq_cleaned)
for i in tesco_prod_name_uniq:
        i1=cleanup(i)
        j=i[i.rstrip().rfind(" "):-2]
        k=[x for x in morrisons_prod_name if x.find(j)!=-1]
        l=np.array(uniq(k))
        if l.size==0:
            ratio = process.extract( i1, morrisons_prod_name_uniq_cleaned, limit=1, scorer=fuzz.partial_token_sort_ratio)
            similarity.append(0)
        else:
            m=np.array([f(xi) for xi in l])
            ratio = process.extract( i1, m, limit=1)
            similarity.append(ratio[0][1])
        res=np.where(morrisons_prod_name_uniq_cleaned==ratio[0][0])
        matched_prod.append(morrisons_prod_name_uniq[res[0][0]])
        
match = pd.DataFrame({'tesco_prod_name':tesco_prod_name_uniq, 'morrisons_prod_name':matched_prod,'similarity':similarity}) 

#print(match.head(1))
match.to_csv('C:/Users/sujithk/Desktop/file1_name.csv')

In [ ]:
def get_matches_df(sparse_matrix, name_vector, top=0):
    non_zeros = sparse_matrix.nonzero()
    
    sparserows = non_zeros[0]
    sparsecols = non_zeros[1]
    
    if top!= 0:
        nr_matches = top
    else:
        nr_matches = sparsecols.size
    print(sparsecols)
    print(sparserows.size)
    Tesco = np.empty([nr_matches], dtype=object) # returns a matrix of given shape
    right_side = np.empty([nr_matches], dtype=object) # returns a matrix of given shape
    similairity = np.zeros(nr_matches) # returns a matrix of given shape with zeros
    i=0
    for index in range(0, nr_matches-1):
        if (sparserows[index] < prim_vec_count) and (sparsecols[index] >= prim_vec_count):
            Tesco[i] = name_vector[sparserows[index]]
            right_side[i] = name_vector[sparsecols[index]]
            similairity[i] = sparse_matrix.data[index]
            i=i+1
            

    
    return pd.DataFrame({'Tesco': Tesco,
                          'right_side': right_side,
                           'similairity': similairity})

matches_df = get_matches_df(matches, test_vector)
matches_df.dropna(subset = ["Tesco"], inplace=True)
print(matches_df.shape)
# print(matches_df.head(100))
matches_df= matches_df.groupby(["Tesco"]).apply(lambda x: x.sort_values(["similairity"], ascending = False)).reset_index(drop=True)
print(matches_df)
matches_df=matches_df.groupby(["Tesco"]).head(1)
matches_df.to_csv('C:/Users/sujithk/Desktop/file_name.csv')
# print(matches_df.sort_values(['similairity'], ascending=False))